In [1]:
#This notebook builds a similarity merger from baseline randomforest 
#Morgan fingerprint 
#and cell painting

In [2]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from scipy.stats import randint

In [3]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


In [4]:
similarity = pd.read_csv("Similarity_def_mean_train_test_no_Classv2.csv")
similarity

,assay,InChICode_standardised,ts,ts_perc,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives,Data
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.290269,0.000000,0.284818,0.000000,0.271792,0.000000,0.859367,0.063584,0.859367,0.209302,0.512414,0.015385,Training
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.787685,0.213873,0.660846,0.093023,0.745801,0.253846,0.754709,0.231214,0.729592,0.465116,0.723072,0.246154,Training
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.772941,0.132948,0.553399,0.023256,0.772941,0.169231,0.557030,0.017341,0.436839,0.046512,0.499498,0.015385,Training
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.812935,0.208092,0.574854,0.069767,0.812935,0.253846,0.717875,0.104046,0.500185,0.023256,0.716548,0.146154,Training
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.770504,0.346821,0.670484,0.255814,0.762533,0.376923,0.478680,0.005780,0.370233,0.000000,0.478680,0.007692,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.324773,0.000000,0.279837,0.000000,0.305871,0.000000,0.681960,0.089109,0.509096,0.111111,0.678402,0.121622,HeldOut
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.316906,0.000000,0.277834,0.000000,0.308875,0.000000,0.504201,0.009901,0.373719,0.037037,0.483532,0.027027,HeldOut
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.276304,0.000000,0.263956,0.000000,0.261232,0.000000,0.425408,0.000000,0.353781,0.000000,0.423772,0.000000,HeldOut
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.343005,0.000000,0.328585,0.000000,0.323085,0.000000,0.628148,0.069307,0.441401,0.074074,0.616700,0.094595,HeldOut


In [5]:
df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv")
df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,True,True,0.858900,0.927333,1.0,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,False,True,0.370592,0.597659,0.0,False
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,True,True,0.048086,0.311150,0.0,True
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,True,True,0.040921,0.339009,0.0,True
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,True,True,0.358348,0.168984,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False,0.260575,0.574224,0.0,True
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True,0.672778,0.376325,1.0,False
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False,0.578650,0.419705,0.0,False
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True,0.529843,0.304008,0.0,True


In [6]:
list_of_lists_df = pd.merge(df, similarity, on=["assay","InChICode_standardised","Data"])
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,...,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,...,0.284818,0.000000,0.271792,0.000000,0.859367,0.063584,0.859367,0.209302,0.512414,0.015385
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,...,0.660846,0.093023,0.745801,0.253846,0.754709,0.231214,0.729592,0.465116,0.723072,0.246154
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,...,0.553399,0.023256,0.772941,0.169231,0.557030,0.017341,0.436839,0.046512,0.499498,0.015385
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,...,0.574854,0.069767,0.812935,0.253846,0.717875,0.104046,0.500185,0.023256,0.716548,0.146154
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,...,0.670484,0.255814,0.762533,0.376923,0.478680,0.005780,0.370233,0.000000,0.478680,0.007692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,...,0.279837,0.000000,0.305871,0.000000,0.681960,0.089109,0.509096,0.111111,0.678402,0.121622
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,...,0.277834,0.000000,0.308875,0.000000,0.504201,0.009901,0.373719,0.037037,0.483532,0.027027
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,...,0.263956,0.000000,0.261232,0.000000,0.425408,0.000000,0.353781,0.000000,0.423772,0.000000
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,...,0.328585,0.000000,0.323085,0.000000,0.628148,0.069307,0.441401,0.074074,0.616700,0.094595


In [7]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
from itertools import product
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict
from collections import Counter
from numpy import argmax

In [8]:
def RF_Fn(assay):

    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    
    
    proba_batch=[]
    pred_batch=[]
    true_batch=[]
    StdInChI_batch=[]
    
    train= detail_list[detail_list["Data"]=="Training"]
    #test
    test= detail_list[detail_list["Data"]=="HeldOut"]
    
    #columns = ["ts", "ts_perc", "ts_actives", "ts_perc_actives", 
    #          "ts_inactives", "ts_perc_inactives", "pc", "pc_perc", "pc_actives", "pc_perc_actives", 
    #          "pc_inactives", "pc_perc_inactives"]
    
    columns = ["ts_actives", "pc_actives", "fp_proba_scaled", "CP_proba_scaled"]
    
    
    print("Model for training data")    

    X_train = train[columns].to_numpy()
    y_train = train["true"].to_numpy()
    X_test = test[columns].to_numpy()
    
    print(len(X_train))
    print(len(X_test))
    
    #print(X_train)
    
    classifier = LogisticRegression(random_state=42, class_weight="balanced")
    classifier.fit(X_train, y_train)
        
    StdInChI = test["InChICode_standardised"].to_numpy()
    pred = classifier.predict(X_test) 
    proba = classifier.predict_proba(X_test)[:,1]
    true = test["true"].to_numpy()
            

    StdInChI_batch = np.concatenate([StdInChI_batch, StdInChI])
    proba_batch = np.concatenate([proba_batch, proba])
    pred_batch = np.concatenate([pred_batch, pred])
    true_batch = np.concatenate([true_batch, true])

    print(len(pred_batch))
    print(len(true_batch))
   
    return StdInChI_batch, true_batch, pred_batch, proba_batch

In [9]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import KFold

def check_heirarchial(assay, method, y_true, y_pred, y_prob):
    
    conf_matrix = confusion_matrix(y_true, y_pred)
    #print(conf_matrix)
    
    #print(classification_report(y_true, y_pred))
    
    ba= balanced_accuracy_score(y_true, y_pred)
    #print('balanced_accuracy_score ',ba)
    
    mcc=matthews_corrcoef(y_true, y_pred)
    #print('matthews_corrcoef ',mcc)
    # Sensitivity, hit rate, recall, or true positive rate
    
    f1= f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    
    #print('F1 Toxic', f1)
    #print('precision', precision)
    
    Specificity = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])
    # Specificity or true negative rate
    Sensitivity = conf_matrix[1,1]/(conf_matrix[1,0]+conf_matrix[1,1]) 
    
    #print( 'Sensitivity', Sensitivity)
    #print( 'Specificity', Specificity)
    AUC = roc_auc_score(y_true, y_prob)
    #print('AUC-ROC ',AUC)    
    # calculate roc curves
    best_model_fpr, best_model_tpr, _ = roc_curve(y_true, y_prob)   
    
    
    row=[assay, method, f1, precision, Sensitivity, Specificity, ba, mcc, AUC]
    metrics_list.append(row)
    
    return metrics_list

In [10]:
import warnings
warnings.filterwarnings("ignore") #ignore, default
import pandas

#sns.set(rc={'figure.figsize':(2, 2)})
#plt.rcParams['figure.dpi'] = 80
#sns.set_style("white")
metrics_list=[]
all_preds_test_compounds=pd.DataFrame(columns=['assay', 'InChICode_standardised', 'fp_proba', 'fp_pred', 'fp_threshold', 'CP_proba',
       'CP_pred', 'CP_threshold', 'true', 'ts', 'pc', 'Data', 'MFP_Correct',
       'CP_Correct', 'fp_proba_scaled', 'CP_proba_scaled', 'simple_pred',
       'Ensemble_Correct', 'merger_pred', 'merger_Correct'])

#for assay in tqdm(assaylist[:7]):
for assay in tqdm(list_of_lists_df.assay.unique()[:]):
    
    print("Assay: ", assay)
    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    detail_list_test = detail_list[detail_list["Data"]=="HeldOut"]

    #print("CP")
    check_heirarchial(assay, "CP", detail_list_test.true, detail_list_test.CP_pred, detail_list_test.CP_proba)
    #sns.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="CP_Correct", legend=False)
    #plt.show()
    
    #print("MFP")
    check_heirarchial(assay, "MFP", detail_list_test.true, detail_list_test.fp_pred, detail_list_test.fp_proba)
    #sns.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="MFP_Correct", legend=False)
    #plt.show()
    
    #print("Ensemble Averaged")
    check_heirarchial(assay, "Ensemble Averaged", detail_list_test.true, detail_list_test.simple_pred, detail_list_test.simple_pred)
    #sns.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="simple_pred", legend=False)
    #plt.show()
    
    #print("Distance Based Merger")
    merger_df=pd.DataFrame()
    
    StdInChI_batch, true_batch, pred_batch, proba_batch = RF_Fn(assay)
    check_heirarchial(assay, "Distance Based Merger", true_batch, pred_batch, proba_batch)
    
    #print(true_batch)
    
    print(assay)
    
    merger_df["InChICode_standardised"]=StdInChI_batch
    merger_df["merger_pred"]= pred_batch
    merger_df["true"]=true_batch
    merger_df["merger_Correct"]= merger_df["merger_pred"] == merger_df["true"]
    merger_df["assay"]=assay
    
    #.scatterplot(data=detail_list[detail_list["true"]==1], x="pc", y="td", hue="herrachial_Correct", legend=False)
    
    #For each assay combine original predictions and merger_df predictions
    #all_preds = pd.DataFrame()
    all_preds_test_compounds= pd.concat([all_preds_test_compounds, pd.merge(detail_list_test, merger_df)])

  6%|██▍                                         | 5/89 [00:00<00:03, 24.75it/s]

Assay:  178_684
Model for training data
217
55
55
55
178_684
Assay:  180_686
Model for training data
163
41
41
41
180_686
Assay:  183_604
Model for training data
144
36
36
36
183_604
Assay:  184_606
Model for training data
249
63
63
63
184_606
Assay:  185_665
Model for training data
96
24
24
24
185_665
Assay:  185_666


 12%|█████▎                                     | 11/89 [00:00<00:02, 27.50it/s]

Model for training data
137
35
35
35
185_666
Assay:  186_668
Model for training data
102
26
26
26
186_668
Assay:  190_628
Model for training data
124
32
32
32
190_628
Assay:  190_629
Model for training data
83
21
21
21
190_629
Assay:  190_674
Model for training data
99
25
25
25
190_674
Assay:  191_630
Model for training data
147
37
37
37
191_630
Assay:  191_631


 19%|████████▏                                  | 17/89 [00:00<00:02, 28.50it/s]

...

Model for training data
1270
318
318
318
268_746
Assay:  269_747
Model for training data
1939
485
485
485
269_747
Assay:  270_748
Model for training data
345
87
87
87
270_748
Assay:  271_749
Model for training data
758
190
190
190
271_749
Assay:  272_750
Model for training data
246
62
62
62
272_750
Assay:  273_751
Model for training data
144
36
36
36
273_751


 93%|████████████████████████████████████████   | 83/89 [00:03<00:00, 26.31it/s]

Assay:  274_752
Model for training data
342
86
86
86
274_752
Assay:  275_753
Model for training data
124
32
32
32
275_753
Assay:  275_754
Model for training data
336
84
84
84
275_754
Assay:  276_755
Model for training data
448
112
112
112
276_755
Assay:  278_757
Model for training data
1206
302
302
302
278_757
Assay:  279_758
Model for training data
211
53


100%|███████████████████████████████████████████| 89/89 [00:03<00:00, 26.63it/s]

53
53
279_758
Assay:  280_759
Model for training data
432
108
108
108
280_759
Assay:  283_762
Model for training data
342
86
86
86
283_762
Assay:  93_259
Model for training data
101
26
26
26
93_259


In [11]:
all_preds_test_compounds

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,ts,...,ts_perc,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives
0,178_684,InChI=1S/C24H28N4O4S/c1-14-12-28(15(2)13-29)24...,0.431564,1,0.420981,0.687784,1,0.306156,1.0,0.777155,...,0.175115,0.713458,0.148148,0.711843,0.184049,0.032258,0.738838,0.129630,0.396936,0.000000
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.431601,1,0.420981,0.186083,0,0.306156,0.0,0.840626,...,0.216590,0.751512,0.092593,0.750294,0.257669,0.000000,0.390391,0.000000,0.446116,0.000000
2,178_684,InChI=1S/C25H35N3O5S/c1-17-10-12-20(13-11-17)3...,0.105376,0,0.420981,0.074175,0,0.306156,0.0,0.801531,...,0.262673,0.642161,0.074074,0.796562,0.325153,0.327189,0.816468,0.370370,0.861748,0.331288
3,178_684,InChI=1S/C31H42N4O6/c1-20-15-35(21(2)18-36)30(...,0.173670,0,0.420981,0.061583,0,0.306156,0.0,0.840029,...,0.225806,0.656758,0.166667,0.821664,0.245399,0.211982,0.686297,0.259259,0.751059,0.288344
4,178_684,InChI=1S/C30H41N5O5/c1-20-17-35(21(2)19-36)28(...,0.307765,0,0.420981,0.083022,0,0.306156,1.0,0.815149,...,0.165899,0.659838,0.074074,0.752292,0.196319,0.000000,0.417694,0.000000,0.515094,0.036810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,0.324773,...,0.000000,0.279837,0.000000,0.305871,0.000000,0.089109,0.509096,0.111111,0.678402,0.121622
22,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,0.316906,...,0.000000,0.277834,0.000000,0.308875,0.000000,0.009901,0.373719,0.037037,0.483532,0.027027
23,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,0.276304,...,0.000000,0.263956,0.000000,0.261232,0.000000,0.000000,0.353781,0.000000,0.423772,0.000000
24,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,0.343005,...,0.000000,0.328585,0.000000,0.323085,0.000000,0.069307,0.441401,0.074074,0.616700,0.094595


In [12]:
all_preds_test_compounds.to_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble_merger_test_compounds.csv", index=False)

In [13]:
metrics_list = pd.DataFrame(metrics_list,columns=["assay","method","f1", "precision", "Sensitivity", "Specificity", "ba", "mcc", "AUC"])
metrics_list = metrics_list.sort_values(by=["assay", "method"]).reset_index(drop=True)
metrics_list

,assay,method,f1,precision,Sensitivity,Specificity,ba,mcc,AUC
0,178_684,CP,0.571429,0.571429,0.571429,0.853659,0.712544,0.425087,0.731707
1,178_684,Distance Based Merger,0.583333,0.700000,0.500000,0.926829,0.713415,0.482063,0.848432
2,178_684,Ensemble Averaged,0.521739,0.666667,0.428571,0.926829,0.677700,0.418479,0.677700
3,178_684,MFP,0.560000,0.636364,0.500000,0.902439,0.701220,0.438262,0.808362
4,180_686,CP,0.400000,0.600000,0.300000,0.935484,0.617742,0.309034,0.716129
...,...,...,...,...,...,...,...,...,...
351,283_762,MFP,0.392523,0.244186,1.000000,0.000000,0.500000,0.000000,0.554579
352,93_259,CP,0.400000,0.375000,0.428571,0.736842,0.582707,0.158970,0.751880
353,93_259,Distance Based Merger,0.461538,0.500000,0.428571,0.842105,0.635338,0.284961,0.766917
354,93_259,Ensemble Averaged,0.307692,0.333333,0.285714,0.789474,0.537594,0.079156,0.537594


In [14]:
metrics_list.to_csv("All_metrics.csv", index=False)